In [11]:
import requests
import os
from pathlib import Path
from dotenv import load_dotenv

#need to update client_keys.env whenever the server key is changed
key_path=  Path('client_key') /'client_keys.env'
load_dotenv(key_path)
header_key = 'House-Price-API-KEY' 
api_key = os.getenv('HOUSE_API_Key')

header_HOUSE_API = {header_key: api_key}

# Define the base URL
BASE_URL = 'http://127.0.0.1:8000'

# Test /health endpoint

expected response: json with status "ok"

should generate a generic logger on server side

In [12]:
print('Making health request')
response = requests.get(url = f'{BASE_URL}/health',headers=header_HOUSE_API)
print ('Response: ',response.json())

Making health request
Response:  {'success': True, 'endpoint': 'health', 'data': {'API status': 'online'}}


# Testing /predict endpoint

- single predict

expected reponse : json with house price

- batch with 2 predictions

expected reponse : json with 2 houses prices

- single predict with abnormal metrics (n_rooms = 3)

expected reponse : json with house price + metrics logger on server side pointing anomaly

all requests above also produces generic logger info on server side

if pred_history logger is enabled (need to comply with privacy settings) it also produces pred_history info logs (input + output + model info)



In [13]:
### Test predict request

## Test single request
print('Making prediction request of a single house')
body=[{
     "type":"casa",
    "sector":"vitacura",
    "net_usable_area":402.0,
    "net_area":257.0,
    "n_rooms":5.0,
    "n_bathroom":2.0,
    "latitude":43.7894,
    "longitude":-105.5837,

}]
response = requests.post(url = f'{BASE_URL}/predict',
              json = body, headers=header_HOUSE_API)
print ('Response: ',response.json())

#Test batch of requests
print('Making prediction request of a batch with 2 houses')
body=[{
    "type":"casa",
    "sector":"vitacura",
    "net_usable_area":183.0,
    "net_area":450.0,
    "n_rooms":4.0,
    "n_bathroom":3.0,
    "latitude":91.8394,
    "longitude":-80.5447,

},
{
    "type":"departamento",
    "sector":"condes",
    "net_usable_area":230.0,
    "net_area":270.0,
    "n_rooms":5.0,
    "n_bathroom":2.0,
    "latitude":43.4235,
    "longitude":50.58,
   
}]

response = requests.post(url = f'{BASE_URL}/predict',
              json = body, headers=header_HOUSE_API)
print ('Response: ',response.json())


print('Making prediction request of a single house with n_rooms = -3 - should get a anomaly logger on server side')
body=[{
     "type":"casa",
    "sector":"vitacura",
    "net_usable_area":383.0,
    "net_area":250.0,
    "n_rooms":7.0,
    "n_bathroom":-3.0,
    "latitude":121.8394,
    "longitude":-130.5447,

}]
response = requests.post(url = f'{BASE_URL}/predict',
              json = body, headers=header_HOUSE_API)
print ('Response: ',response.json())




Making prediction request of a single house
Response:  {'success': True, 'endpoint': 'predict', 'data': {'price': [23964.962250699562]}}
Making prediction request of a batch with 2 houses
Response:  {'success': True, 'endpoint': 'predict', 'data': {'price': [18650.30407076638, 19365.280316261164]}}
Making prediction request of a single house with n_rooms = -3 - should get a anomaly logger on server side
Response:  {'success': True, 'endpoint': 'predict', 'data': {'price': [23510.520496190584]}}


In [14]:
print('Making prediction missing type value. type is an obrigatory field.')
body=[{
     "type":None,
    "sector":"vitacura",
    "net_usable_area":152.0,
    "net_area":200,
    "n_rooms":-3.0,
    "n_bathroom":4.0,
    "latitude":53.3794,
    "longitude":-80.5447,

}]
response = requests.post(url = f'{BASE_URL}/predict',
              json = body, headers=header_HOUSE_API)
print ('Response: ',response.json())

Making prediction missing type value. type is an obrigatory field.
Response:  {'detail': [{'type': 'string_type', 'loc': ['body', 0, 'type'], 'msg': 'Input should be a valid string', 'input': None, 'url': 'https://errors.pydantic.dev/2.5/v/string_type'}]}


In [15]:
print('Making prediction missing net_area value. Net_area is an optional field')
body=[{
     "type":"casa",
    "sector":"vitacura",
    "net_usable_area":383.0,
    "net_area":None,
    "n_rooms":7.0,
    "n_bathroom":-3.0,
    "latitude":121.8394,
    "longitude":-130.5447,

}]
response = requests.post(url = f'{BASE_URL}/predict',
              json = body, headers=header_HOUSE_API)
print ('Response: ',response.json())

Making prediction missing net_area value. Net_area is an optional field
Response:  {'success': True, 'endpoint': 'predict', 'data': {'price': [30461.51050397201]}}


# Testing /info endpoint

expected response: json with model name, version, filename and last_edited date

In [16]:
print('Making info request')
response = requests.get(url = f'{BASE_URL}/info',headers=header_HOUSE_API)
print ('Response: ',response.json())

Making info request
Response:  {'success': True, 'endpoint': 'info', 'data': {'name': 'House_Price', 'version': '1', 'alias': 'production'}}
